In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D, proj3d
import numpy as np
from ipycanvas import Canvas, hold_canvas
from ipywidgets import FloatSlider, Output
from math import pi

In [ ]:
out = Output(layout={'border': '1px solid black'})

In [ ]:
class Plot3d(Canvas):
    def __init__(self):
        super(Plot3d, self).__init__(size=(500, 500))
        
        self.width = 500
        self.height = 500
        
        plt.ioff()
        fig = plt.figure()
        self.ax = Axes3D(fig)

        self.dragging = False
        self.n = 200
        self.x = np.random.rand(self.n)
        self.y = np.random.rand(self.n)
        self.z = np.random.rand(self.n)
        
        self.zoom = 4
        self.dx = 0
        self.dy = 0
        self.ax.view_init(elev=self.dx, azim=self.dy)
        self.x2, self.y2, _ = proj3d.proj_transform(self.x, self.y, self.z, self.ax.get_proj())
        self.draw()
            
        self.on_mouse_down(self.mouse_down_handler)
        self.on_mouse_move(self.mouse_move_handler)
        self.on_mouse_up(self.mouse_up_handler)
        self.on_mouse_out(self.mouse_out_handler)

    #@out.capture()
    def draw(self):
        x = self.x2 * self.width * self.zoom + self.width / 2
        y = self.y2 * self.width * self.zoom + self.height / 2
        with hold_canvas(self):
            self.clear()
            self.save()
            for i in range(self.n):
                self.fill_arc(x[i], y[i], self.zoom, 0, 2*pi)
                self.stroke_arc(x[i], y[i], self.zoom, 0, 2*pi)
            self.restore()

    def mouse_down_handler(self, pixel_x, pixel_y):
        self.dragging = True
        self.x_mouse = pixel_x
        self.y_mouse = pixel_y

    def mouse_move_handler(self, pixel_x, pixel_y):
        if self.dragging:
            self.dx_new = self.dx + pixel_x - self.x_mouse
            self.dy_new = self.dy + pixel_y - self.y_mouse
            self.ax.view_init(elev=self.dy_new, azim=self.dx_new)
            self.x2, self.y2, _ = proj3d.proj_transform(self.x, self.y, self.z, self.ax.get_proj())
            self.draw()
    
    def mouse_up_handler(self, pixel_x, pixel_y):
        if self.dragging:
            self.dragging = False
            self.dx = self.dx_new
            self.dy = self.dy_new
    
    def mouse_out_handler(self, pixel_x, pixel_y):
        if self.dragging:
            self.dragging = False
            self.dx = self.dx_new
            self.dy = self.dy_new

In [ ]:
p = Plot3d()
p

In [ ]:
slider = FloatSlider(description='Zoom:', value=p.zoom, min=1, max=10)

def on_slider_change(change):
    p.zoom = slider.value
    p.draw()

slider.observe(on_slider_change, 'value')
slider

## Drag the figure with the mouse to rotate, use the slider to zoom

In [ ]:
out